In this notebook we:
- Download the feature vectors/reduced dim feature vectors from S3 (7 options)
- Get the distance matrices for each of these
- Pick the nodes you are going to go between in the network
- Build the graphs using 3 types of neighbour definitions (top neighbours, or neighbours close defined by a threshold, a mixture of these or a fully connected graph)
- Run different pathways (dijkstra path, the a* path or my defined path) using these graphs

The outcome of this notebook points to using the __raw feature vectors, and with a network where each node is connected to its top 3 neighbours__.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm
import os
from io import BytesIO
import ast
import numpy as np
import pickle
from itertools import compress
from collections import Counter
import operator
from functools import partial

from PIL import Image
import torch
import boto3
from scipy.spatial.distance import cdist
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from itertools import combinations
import umap.umap_ as umap

In [ ]:
cd ..

In [ ]:
from src.network_functions import (
    import_feature_vectors,
    get_all_s3_keys,
    get_distances,
    image_pathway_plot,
    get_top_neighbours,
    get_high_neighbours,
    get_top_high_neighbours,
    create_network_graph,
    plot_graph,
    defined_path,
)

In [ ]:
images_dir = "data/"
image_type = ".png"

### 1. Get the names of the ~5000 feature vectors which I found different dimensionality reductions

Pick a sample if you want to make it quicker

In [ ]:
bucket_name = "miro-images-feature-vectors"
bucket_name = bucket_name
s3 = boto3.client("s3")

In [ ]:
keys = get_all_s3_keys(bucket_name, s3)

In [ ]:
folder_name = "reduced_feature_vectors_100_dims"

image_names = [os.path.split(k)[1] for k in keys if folder_name in k]

In [ ]:
n_sample = 1000
np.random.seed(0)  # For dev
image_names = np.random.choice(image_names, n_sample, replace=False)

In [ ]:
len(image_names)

### 2. Download the feature vectors/reduced dim feature vectors from S3

In [ ]:
feature_vectors, _ = import_feature_vectors(
    s3, bucket_name, "feature_vectors", image_names
)
feature_vectors_2_dims, _ = import_feature_vectors(
    s3, bucket_name, "reduced_feature_vectors_2_dims", image_names
)
feature_vectors_20_dims, _ = import_feature_vectors(
    s3, bucket_name, "reduced_feature_vectors_20_dims", image_names
)
feature_vectors_80_dims, _ = import_feature_vectors(
    s3, bucket_name, "reduced_feature_vectors_80_dims", image_names
)
feature_vectors_100_dims, _ = import_feature_vectors(
    s3, bucket_name, "reduced_feature_vectors_100_dims", image_names
)
feature_vectors_500_dims, _ = import_feature_vectors(
    s3, bucket_name, "reduced_feature_vectors_500_dims", image_names
)
feature_vectors_1000_dims, _ = import_feature_vectors(
    s3, bucket_name, "reduced_feature_vectors_1000_dims", image_names
)

In [ ]:
# Remove the name of this image from the list if no feature vector data was found for it
image_names = [x for x in image_names if x in list(feature_vectors.keys())]
image_names = [x for x in image_names if x in list(feature_vectors_100_dims.keys())]
len(image_names)

In [ ]:
image_names_dict = {k: v for k, v in enumerate(image_names)}

### 3. Get the distance matrices

In [ ]:
dist_mat_fv = get_distances(feature_vectors)
dist_mat_fv2 = get_distances(feature_vectors_2_dims)
dist_mat_fv20 = get_distances(feature_vectors_20_dims)
dist_mat_fv80 = get_distances(feature_vectors_80_dims)
dist_mat_fv100 = get_distances(feature_vectors_100_dims)
dist_mat_fv500 = get_distances(feature_vectors_500_dims)
dist_mat_fv1000 = get_distances(feature_vectors_1000_dims)

### 4. To save running time, build the graphs, then mess with pathway algos.

I build four types of graphs using the parameters (when applicable):
- number_neighbours = 3
- dist_threshold = 0.35

Types of graphs:
1. Using the top n neighbours : each node is connected to its n closest neighbours
2. Using all connections < threshold distance : each node is connected to all it's closest neighbours, defined by a threshold
3. Using all connections < threshold distance or top n : each node is connected to all it's closest neighbours, defined by a threshold, and if there are no 'close' neighbours, then the top n
4. Fully connected graph : every node is connected to each other


In [ ]:
def run_graph(
    dist_mat, neighbour_function, number_neighbours=None, dist_threshold=None
):

    if neighbour_function == get_top_neighbours:
        dist_mat_neighbours = neighbour_function(dist_mat=dist_mat, n=number_neighbours)
    elif neighbour_function == get_high_neighbours:
        dist_mat_neighbours = neighbour_function(
            dist_mat=dist_mat, dist_threshold=dist_threshold
        )
    elif neighbour_function == get_top_high_neighbours:
        dist_mat_neighbours = neighbour_function(
            dist_mat=dist_mat, n=number_neighbours, dist_threshold=dist_threshold
        )

    G = create_network_graph(dist_mat_neighbours)

    return G

In [ ]:
neighbour_function = get_top_neighbours
number_neighbours = 3

run_graph_partial = partial(
    run_graph,
    neighbour_function=neighbour_function,
    number_neighbours=number_neighbours,
)

G_top_fv = run_graph_partial(dist_mat_fv)
G_top_fv2 = run_graph_partial(dist_mat_fv2)
G_top_fv20 = run_graph_partial(dist_mat_fv20)
G_top_fv80 = run_graph_partial(dist_mat_fv80)
G_top_fv100 = run_graph_partial(dist_mat_fv100)
G_top_fv500 = run_graph_partial(dist_mat_fv500)
G_top_fv1000 = run_graph_partial(dist_mat_fv1000)

In [ ]:
G_top_dict = {
    "G_top_fv": G_top_fv,
    "G_top_fv2": G_top_fv2,
    "G_top_fv20": G_top_fv20,
    "G_top_fv80": G_top_fv80,
    "G_top_fv100": G_top_fv100,
    "G_top_fv500": G_top_fv500,
    "G_top_fv1000": G_top_fv1000,
}

In [ ]:
_ = plot_graph(G_top_fv, figsize=(3, 3))
_ = plot_graph(G_top_fv2, figsize=(3, 3))
_ = plot_graph(G_top_fv20, figsize=(3, 3))
_ = plot_graph(G_top_fv80, figsize=(3, 3))
_ = plot_graph(G_top_fv100, figsize=(3, 3))
_ = plot_graph(G_top_fv500, figsize=(3, 3))
_ = plot_graph(G_top_fv1000, figsize=(3, 3))

In [ ]:
neighbour_function = get_high_neighbours
dist_threshold = 0.8

run_graph_partial = partial(
    run_graph, neighbour_function=neighbour_function, dist_threshold=dist_threshold
)

G_high_fv = run_graph_partial(dist_mat_fv)
G_high_fv2 = run_graph_partial(dist_mat_fv2)
G_high_fv20 = run_graph_partial(dist_mat_fv20)
G_high_fv80 = run_graph_partial(dist_mat_fv80)
G_high_fv100 = run_graph_partial(dist_mat_fv100)
G_high_fv500 = run_graph_partial(dist_mat_fv500)
G_high_fv1000 = run_graph_partial(dist_mat_fv1000)

In [ ]:
G_high_dict = {
    "G_high_fv": G_high_fv,
    "G_high_fv2": G_high_fv2,
    "G_high_fv20": G_high_fv20,
    "G_high_fv80": G_high_fv80,
    "G_high_fv100": G_high_fv100,
    "G_high_fv500": G_high_fv500,
    "G_high_fv1000": G_high_fv1000,
}

In [ ]:
_ = plot_graph(G_high_fv, figsize=(3, 3))
_ = plot_graph(G_high_fv2, figsize=(3, 3))
_ = plot_graph(G_high_fv20, figsize=(3, 3))
_ = plot_graph(G_high_fv80, figsize=(3, 3))
_ = plot_graph(G_high_fv100, figsize=(3, 3))
_ = plot_graph(G_high_fv500, figsize=(3, 3))
_ = plot_graph(G_high_fv1000, figsize=(3, 3))

In [ ]:
neighbour_function = get_top_high_neighbours
dist_threshold = 0.8
number_neighbours = 3

run_graph_partial = partial(
    run_graph,
    neighbour_function=neighbour_function,
    number_neighbours=number_neighbours,
    dist_threshold=dist_threshold,
)

G_tophigh_fv = run_graph_partial(dist_mat_fv)
G_tophigh_fv2 = run_graph_partial(dist_mat_fv2)
G_tophigh_fv20 = run_graph_partial(dist_mat_fv20)
G_tophigh_fv80 = run_graph_partial(dist_mat_fv80)
G_tophigh_fv100 = run_graph_partial(dist_mat_fv100)
G_tophigh_fv500 = run_graph_partial(dist_mat_fv500)
G_tophigh_fv1000 = run_graph_partial(dist_mat_fv1000)

In [ ]:
G_tophigh_dict = {
    "G_tophigh_fv": G_tophigh_fv,
    "G_tophigh_fv2": G_tophigh_fv2,
    "G_tophigh_fv20": G_tophigh_fv20,
    "G_tophigh_fv80": G_tophigh_fv80,
    "G_tophigh_fv100": G_tophigh_fv100,
    "G_tophigh_fv500": G_tophigh_fv500,
    "G_tophigh_fv1000": G_tophigh_fv1000,
}

In [ ]:
_ = plot_graph(G_tophigh_fv, figsize=(3, 3))
_ = plot_graph(G_tophigh_fv2, figsize=(3, 3))
_ = plot_graph(G_tophigh_fv20, figsize=(3, 3))
_ = plot_graph(G_tophigh_fv80, figsize=(3, 3))
_ = plot_graph(G_tophigh_fv100, figsize=(3, 3))
_ = plot_graph(G_tophigh_fv500, figsize=(3, 3))
_ = plot_graph(G_tophigh_fv1000, figsize=(3, 3))

In [ ]:
# Fully connected graphs
G_full_fv = create_network_graph(dist_mat_fv)
G_full_fv2 = create_network_graph(dist_mat_fv2)
G_full_fv20 = create_network_graph(dist_mat_fv20)
G_full_fv80 = create_network_graph(dist_mat_fv80)
G_full_fv100 = create_network_graph(dist_mat_fv100)
G_full_fv500 = create_network_graph(dist_mat_fv500)
G_full_fv1000 = create_network_graph(dist_mat_fv1000)

In [ ]:
G_full_dict = {
    "G_full_fv": G_full_fv,
    "G_full_fv2": G_full_fv2,
    "G_full_fv20": G_full_fv20,
    "G_full_fv80": G_full_fv80,
    "G_full_fv100": G_full_fv100,
    "G_full_fv500": G_full_fv500,
    "G_full_fv1000": G_full_fv1000,
}

In [ ]:
_ = plot_graph(G_full_fv, figsize=(3, 3))
_ = plot_graph(G_full_fv80, figsize=(3, 3))
_ = plot_graph(G_full_fv1000, figsize=(3, 3))

### 5. Pick the nodes you are going to go between in the network

- Furthest apart? High cosine distance = different image features
- Random?

In [ ]:
high_coords = np.where(dist_mat_fv == np.amax(dist_mat_fv))
print(
    "Picking the first highest cosine out of {} with the same highest value".format(
        len(high_coords)
    )
)
node1 = list(zip(high_coords[0], high_coords[1]))[0][0]
node2 = list(zip(high_coords[0], high_coords[1]))[0][1]
print(node1)
print(node2)
print(image_names_dict[node1])
print(image_names_dict[node2])

In [ ]:
np.random.seed(4)
node1 = np.random.choice(list(image_names_dict))
node2 = np.random.choice(list(image_names_dict))
print(node1)
print(node2)
print(image_names_dict[node1])  # V0040357EL
print(image_names_dict[node2])  # V0020158

# Run different pathways using these graphs

In [ ]:
def run_pathway(
    G_dict,
    pathway_algo,
    node1,
    node2,
    image_names_dict,
    images_dir,
    image_type,
    path_size=None,
    best_path=True,
    best_type="sum",
):

    G = G_dict[1]
    try:
        if pathway_algo == nx.dijkstra_path:
            node_path = pathway_algo(G, node1, node2, weight=None)
        elif pathway_algo == nx.astar_path:
            node_path = pathway_algo(G, node1, node2, weight=None)
        elif pathway_algo == defined_path:
            G_weights = nx.to_numpy_matrix(G)
            node_path = pathway_algo(
                G, node1, node2, G_weights, path_size, best_path, best_type
            )

        image_names_path = [image_names_dict[n] for n in node_path]

        title = "Graph type is {}.\nPathway algo is {}.\nBest type is {}".format(
            G_dict[0], str(locals()["pathway_algo"]), best_type
        )

        return (
            image_pathway_plot(images_dir, image_type, image_names_path, title),
            node_path,
        )
    except:
        return print("There is no pathway between nodes"), _

## Play with the dijkstra_path pathway

In [ ]:
pathway_algo = nx.dijkstra_path

run_pathway_partial = partial(
    run_pathway,
    pathway_algo=pathway_algo,
    node1=node1,
    node2=node2,
    image_names_dict=image_names_dict,
    images_dir=images_dir,
    image_type=image_type,
)

In [ ]:
for G_top in G_top_dict.items():
    run_pathway_partial(G_top)

In [ ]:
for G_high in G_high_dict.items():
    run_pathway_partial(G_high)

In [ ]:
for G_tophigh in G_tophigh_dict.items():
    run_pathway_partial(G_tophigh)

Try using my defined path function. In this I can use the fully connected graph too. Note that using the fully connected graph with an undefined number of nodes will just return a direct pathway from the first image to the second.

### Play with the A* path

In [ ]:
pathway_algo = nx.astar_path

run_astar_pathway_partial = partial(
    run_pathway,
    pathway_algo=pathway_algo,
    node1=node1,
    node2=node2,
    image_names_dict=image_names_dict,
    images_dir=images_dir,
    image_type=image_type,
)

In [ ]:
for G_top in G_top_dict.items():
    run_astar_pathway_partial(G_top)

In [ ]:
run_astar_pathway_partial(("G_full_fv", G_full_dict["G_full_fv"]))

### Play with the defined_path

In [ ]:
pathway_algo = defined_path

run_defined_pathway_partial = partial(
    run_pathway,
    pathway_algo=pathway_algo,
    node1=node1,
    node2=node2,
    image_names_dict=image_names_dict,
    images_dir=images_dir,
    image_type=image_type,
)

In [ ]:
run_defined_pathway_partial(
    ("G_top_fv", G_top_dict["G_top_fv"]), path_size=10, best_type="sum"
)

In [ ]:
run_defined_pathway_partial(
    ("G_top_fv", G_top_dict["G_top_fv"]), path_size=10, best_type="average"
)

In [ ]:
run_defined_pathway_partial(
    ("G_top_fv", G_top_dict["G_top_fv"]), path_size=10, best_type="variance"
)

In [ ]:
for G_top in G_top_dict.items():
    run_defined_pathway_partial(G_top, path_size=9, best_type="sum")
    run_defined_pathway_partial(G_top, path_size=9, best_type="variance")

In [ ]:
run_defined_pathway_partial(
    ("G_full_fv", G_full_dict["G_full_fv"]), path_size=3, best_type="variance"
)

In [ ]:
# Takes so long!
# run_defined_pathway_partial(('G_full_fv', G_full_dict['G_full_fv']), path_size=5)